In [51]:
!cat bibliotecas.txt | xargs -n 1 pip3 install

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached https://www.piwheels.org/simple/rpi-gpio/RPi.GPIO-0.7.0-cp37-cp37m-linux_armv7l.whl (69 kB)
You should consider upgrading via the '/home/pi/.virtualenvs/tcc/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached https://www.piwheels.org/simple/rpi-gpio/RPi.GPIO-0.7.0-cp37-cp37m-linux_armv7l.whl (69 kB)
You should consider upgrading via the '/home/pi/.virtualenvs/tcc/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached https://www.piwheels.org/simple/rpi-gpio/RPi.GPIO-0.7.0-cp37-cp37m-linux_armv7l.whl (69 kB)
You should consider upgrading via the '/home/pi/.virtualenvs/tcc/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
You should consider upgrad

In [53]:
#Importação de bibliotecas
import RPi.GPIO as GPIO
import time, copy
import board, busio, adafruit_vl53l0x
import serial
import numpy as np

In [2]:
#Setup das portas logicas do Raspibery PI
class SetupGPIO:
    def __init__(self):
        self.objGPIO = GPIO
        self.objGPIO.cleanup() # Liaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa/*****************************************************************************/********                 mpa todos os estados de todas as portas
        self.objGPIO.setmode(GPIO.BCM) #Definindi uso dos numeros das portas por canais
    
    def get_gpio(self):
        return self.objGPIO

In [17]:
class Gsm_GPS:
    def __init__(self):
        self.port=serial.Serial("/dev/tty8",baudrate=11500, timeout=5  )
        self.port.flush()
        self.habilita_gsm_gps()
    
    def habilita_gsm_gps(self):
        self.port.write('AT\r'.encode())
        self.response()
        self.port.write('AT+CSQ\r'.encode())
        self.response()
        port.write('AT+CPIN?\r')
        response()
    
    def response(self):
        responseCount = 0
        while True:
            resp = self.port.readline().decode()
            print(resp)
            print(responseCount)
            if "OK" in resp:
                break
            if "ERROR" in resp:
                break
            if responseCount > 30:
                break
            else:
                responseCount +=1

In [16]:
#gsmgps = Gsm_GPS()

SerialException: [Errno 13] could not open port /dev/tty8: [Errno 13] Permission denied: '/dev/tty8'

In [46]:
#movimentação do servo motor

class StepMotor:
    def __init__(self, raspGPIO, radar):
        self.controlPinEsquerda = [26,19,13,6]
        self.controlPinDireita = [6,13,19,26]
        self.posicoesMedicaRadar = [0,50, 100, 150, 200, 250, 299]
        self.medicaoRadar=[]
        self.raspGPIO = raspGPIO
        self.posicao_final = 0 #Esquerda
        
        self.seq1 = [[1,0,0,0], 
               [1,1,0,0],
               [0,1,0,0],
               [0,1,1,0],
               [0,0,1,0],
               [0,0,1,1],
               [0,0,0,1],
               [1,0,0,1]]
        self.seq2 = [
                [1,0,0,1],
                [1,0,0,0], 
                [1,1,0,0],
                [0,1,0,0],
                [0,1,1,0],
                [0,0,1,0],
                [0,0,1,1],
                [0,0,0,1]]
        self.parar_motor();

    def parar_motor(self):
        for pin in self.controlPinEsquerda:
            raspGPIO.setup(pin, raspGPIO.OUT)
            raspGPIO.output(pin, False)    
    
    def moverMotorEsquerda(self):
        self.medicaoRadar=[]
        for i in range(0, 300,1):
            if i in self.posicoesMedicaRadar:
                self.medicaoRadar.append(radar.get_radar_posicao())
                        
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinEsquerda[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)

    def moverMotorDireita(self):
        self.medicaoRadar=[]
        for i in range(0,300,1):
            if i in self.posicoesMedicaRadar:
                self.medicaoRadar.append(radar.get_radar_posicao())
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinDireita[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
    

    def moverMotorDireita_ajuste(self, valor):
        for i in range(0,valor,1):
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinDireita[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
       
    def moverMotorEsquerda_ajuste(self, valor):
        for i in range(0, valor, 1):
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinEsquerda[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
        
    def get_posicoes_radar(self):
        if self.posicao_final == 0:
            self.moverMotorEsquerda()
            self.posicao_final = 1
        else:
            self.moverMotorDireita()
            self.posicao_final = 0
            self.medicaoRadar = self.medicaoRadar[::-1] 
            
        return self.medicaoRadar

In [47]:
class Vl53_Radar:
    def __init__(self):
        self.i2c = busio.I2C(board.SCL, board.SDA)
        self.vl53 = adafruit_vl53l0x.VL53L0X(self.i2c)
        #self.fig = plt.figure()
        #self.ax1 = fig.add_subplot(1,1,1)
        #self.xarr = []
        self.yarr = []
        self.count = 0

    
    def get_radar(self):
        return self.vl53
    
    def get_radar_posicao(self):
        return int(self.vl53.range / 10)

In [48]:

raspGPIO  = SetupGPIO().get_gpio()
radar     = Vl53_Radar()
stepMotor = StepMotor(raspGPIO, radar)

In [49]:
 for i in range(0,2,1):
     try:
        print(stepMotor.get_posicoes_radar())
     finally:
        stepMotor.parar_motor()


[87, 54, 54, 819, 819, 819, 50]
[86, 52, 56, 819, 819, 39, 51]


In [39]:
class Hiperparametros():
    def __init__(self):
        self.steps = 1000
        self.epsodes = 1000
        self.lr = 0.02
        self.directions = 6
        self.best_directions = 6
        assert self.best_directions <= self.directions
        self.noise = 0.03
        self.seed = 1
        self.env_nome = ''


In [55]:
#normalizacao dos estados (Standardization)
class Normalizacao():
    def __init__(self, inputs):
        self.n = np.zeros(inputs)
        self.mean = np.zeros(inputs)
        self.mean_diff = np.zeros(inputs)
        self.var = np.zeros(inputs)
    
    def observe(self, x):
        self.n +=1.
        last_mean = self.mean.copy()
        self.mean += (x - self.mean) / self.n
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean_diff/self.n).clip(min = 1e-2) #minino 0.01

    def normalize(self, inputs):
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        return (inputs - obs_mean) / obs_std

In [2]:
class Politicas():
    def __init__(self, input_size, output_size):
        self.theta = np.zeros((output_size, input_size))
    
    def evaluate(self, input, delta=None, direction=None):
        if direction is None:
            return self.theta.dot(input)
        elif direction == 'positive':
            return (self.theta + hp.noise * delta).dot(input)
        else: 
            return (self.theta - hp.noise * delta).dot(input)
    
    def sample_deltas(self):
        return [np.random.randn(*self.theta.shape) for _ in range(hp.directions)]
    
    def update(self, rollouts, sigma_r):
        step = np.zeros(self.theta.shape)
        for r_pos, r_neg, d in rollouts:
            step == (r_pos - r_neg) * d
        self.theta += hp.learning_rate / (hp.best_directions * sigma_r) * step

In [3]:
def explore(env, normalizer, policy, direction = None, delta=None):
    state = env.reset() #ler o ambiente radar
    done = False
    num_plays = 0.
    sum_rewards = 0
    while not done and num_plays < Hiperparametros.epsodes:
       normalizer.observe(state)
       state = normalizer.normalize(state)
       action = policy.evaluate(state, delta, direction)
       state. reward, done, _ = env.step(action) #aquifazer o agente se mover
       reward = max(min(reward, 1), -1)#evita outlier nas recompensas
       sum_rewards += reward
       num_plays =+1
    return sum_rewards


In [6]:
#Treinando
def train(env, policy, normalizer, hp):
    for step in range(hp.steps):
        #Inicializacao das pertubacoes (deltas) e as recompensas negativas e positivas)
        deltas = policy.samples_deltas()
        positive_rewards = [0] * hp.directions
        negative_rewards = [0] * hp.directions

        #obtendo as direcao positiva
        for k in range(hp.directions):
            positive_rewards[k] = explore(env, normalizer, policy, direction='positive', delta=deltas[k])

        #obtendo recompensa na direcao negativa
        for k in range(hp.directions):
            positive_rewards[k] = explore(env, normalizer, policy,  direction='negative', delta=deltas[k])
        
        #obtendo todas as recompensas positivas e negativas para computar o desvio dessas recompensas
        all_reward = np.array(positive_rewards + negative_rewards)
        sigma_r = all_rewards.std()

        #ordenacao dos rollouts e selecao das melhores direcoes
        scores = {k: max(r_pos, r_neg) for k, (r_pos, r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
        order = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)[:hp.best_directions]
        rollouts = [(positive_rewards[k], negative_rewards[k], deltas [k]) for k in order]
        #atualizacao de politica
        policy.update(rollouts, sigma_r)

        #impressao da recompensa
        reward_evaluation = explore(env, normalizer, policy)
        print('Step', step, 'Reward:',reward_evaluation )
             

In [ ]:
 hp = Hiperparametros()
 np.random.seed(hp.seed)

train(,,,hpkkkk dei)